## 1. Install gcloud CLI Tool

If you are running this notebook on GCP, the `gcloud` CLI tool is likely already pre-installed. However, if you are running into issue accessing `gcloud` command, first check your `$PATH` in the shell environment. If issue persists, you can use the following command to download the latest packages onto your environment.

In jupyter notebook cell:
```shell
!curl -sSL https://sdk.cloud.google.com | bash
!exec -l $SHELL
```

In terminal shell:

```shell
curl -sSL https://sdk.cloud.google.com | bash
exec -l $SHELL
```

#### 1.2 Update the gcloud component
We will use `gcloud components update` to update the ***gcloud*** command



!gcloud components update

## 2. Setup API
We will perform the following action to setup the GenAI API for our python code use in the later portion.

```shell
#!/bin/bash

# Set variables
PROJECT_ID="your-gcp-project-id" # replace with your project id
SECRET_NAME="gemini-api-key" # set secret name in Secret Manager
DISPLAY_NAME="Gemini-API-Key" # Set API key display name

# Step 1: Create the API key
echo "Creating API key..."
API_KEY=$(gcloud services api-keys create --display-name="$DISPLAY_NAME" --project="$PROJECT_ID" --format="value(keyString)")

if [ -z "$API_KEY" ]; then
  echo "Failed to create API key."
  exit 1
fi
echo "API key created successfully."

# Step 2: Create a secret in Secret Manager
echo "Creating secret in Secret Manager..."
gcloud secrets create "$SECRET_NAME" --replication-policy="automatic" --project="$PROJECT_ID" || {
  echo "Failed to create secret. Secret may already exist."
}

# Step 3: Add the API key to the secret
echo "Adding API key to the secret..."
echo -n "$API_KEY" | gcloud secrets versions add "$SECRET_NAME" --data-file=- --project="$PROJECT_ID" || {
  echo "Failed to add the API key to the secret."
  exit 1
}
echo "API key successfully added to Secret Manager."

# Step 4: Verify the secret
echo "Verifying the stored secret..."
STORED_KEY=$(gcloud secrets versions access latest --secret="$SECRET_NAME" --project="$PROJECT_ID")

if [ "$STORED_KEY" == "$API_KEY" ]; then
  echo "Secret verified successfully. API key matches the stored secret."
else
  echo "Secret verification failed. Stored secret does not match the API key."
  exit 1
fi

echo "All steps completed successfully!"
```

In [ ]:
import subprocess

# Define the Bash script
bash_script = """
#!/bin/bash

# Set variables
PROJECT_ID="your-gcp-project-id" # replace with your project id
SECRET_NAME="gemini-api-key" # set secret name in Secret Manager
DISPLAY_NAME="Gemini-API-Key" # Set API key display name

# Step 1: Create the API key
echo "Creating API key..."
API_KEY=$(gcloud services api-keys create --display-name="$DISPLAY_NAME" --project="$PROJECT_ID" --format="value(keyString)")

if [ -z "$API_KEY" ]; then
  echo "Failed to create API key."
  exit 1
fi
echo "API key created successfully."

# Step 2: Create a secret in Secret Manager
echo "Creating secret in Secret Manager..."
gcloud secrets create "$SECRET_NAME" --replication-policy="automatic" --project="$PROJECT_ID" || {
  echo "Failed to create secret. Secret may already exist."
}

# Step 3: Add the API key to the secret
echo "Adding API key to the secret..."
echo -n "$API_KEY" | gcloud secrets versions add "$SECRET_NAME" --data-file=- --project="$PROJECT_ID" || {
  echo "Failed to add the API key to the secret."
  exit 1
}
echo "API key successfully added to Secret Manager."

# Step 4: Verify the secret
echo "Verifying the stored secret..."
STORED_KEY=$(gcloud secrets versions access latest --secret="$SECRET_NAME" --project="$PROJECT_ID")

if [ "$STORED_KEY" == "$API_KEY" ]; then
  echo "Secret verified successfully. API key matches the stored secret."
else
  echo "Secret verification failed. Stored secret does not match the API key."
  exit 1
fi

echo "All steps completed successfully!"
"""

# Write the script to a file
script_path = "create_api_key_and_store.sh"
with open(script_path, "w") as script_file:
    script_file.write(bash_script)

# Make the script executable
subprocess.run(["chmod", "+x", script_path], check=True)

# Run the script
try:
    result = subprocess.run(["bash", script_path], check=True, text=True, capture_output=True)
    print(result.stdout)  # Output the script's stdout
except subprocess.CalledProcessError as e:
    print(f"Script failed with error: {e.stderr}")

## Run below cell to install python library dependencies

In [ ]:
!pip install google-generativeai google-cloud-secret-manager

In [ ]:
from IPython import get_ipython
from IPython.display import Markdown, display

In [ ]:
! gcloud services enable secretmanager.googleapis.com

In [ ]:
import subprocess
import json
import pandas as pd
def get_projects_dataframe():
    # Command to get projects in JSON format
    command = ["gcloud", "projects", "list", "--format=json"]

    # Run the command and capture the output
    result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

    # Check for errors
    if result.returncode != 0:
        print("An error occurred while running gcloud projects list:")
        print(result.stderr)
        exit(1)

    # Parse the JSON output
    projects_data = json.loads(result.stdout)

    # Convert to pandas DataFrame
    df = pd.DataFrame(projects_data)

    return df

# Get the DataFrame
df = get_projects_dataframe()

# Display the DataFrame
projectNumber = df['projectNumber']
print(projectNumber)

In [ ]:
! gcloud secrets create gemini_api \
    --replication-policy="automatic"

**Now fetch your API key for Gemini from [this website](https://aistudio.google.com/app/apikey), click `Get API Key`.** and add it to your secret manager secret.

In [ ]:
from google.cloud import secretmanager

def access_secret_version(resource_name):
    client = secretmanager.SecretManagerServiceClient()
    response = client.access_secret_version(request={"name": resource_name})
    return response.payload.data.decode("UTF-8")


# Your secret's resource name
resource_name = f"projects/{projectNumber[0]}/secrets/gemini_api/versions/1"

# Access the secret
apiKey = access_secret_version(resource_name)

In [ ]:
import google.generativeai as genai

genai.configure(api_key=apiKey)

model = genai.GenerativeModel("gemini-1.5-flash")
# response = model.generate_content("Explain how AI works")
# print(response.text)
welcome_prompt = "You are a excellent developer in life science and healthcare research. The mission is to advise researchers with limited coding experience. Please format your response in markdown by default."

In [ ]:
def explain(cell_number):
    """Return the content of the specified cell number."""
    ipython = get_ipython()  # Get the current IPython instance
    
    # Ensure the cell number is valid
    if cell_number < len(ipython.user_ns['In']):
        content = "Based on the input. Please concisely comment this code to explain each line. Ignore the run cell magic and just focus on the code or error " + ipython.user_ns['In'][cell_number] + "Do not add additional code"
        response = model.generate_content(welcome_prompt + content)

        return display(Markdown(response.text))
        # Return the content of the cell
        #return ipython.user_ns['In'][cell_number]
    else:
        # Error message for invalid cell number
        return "Cell number out of range."
    

    
def modify(cell_number,modification):
    """Return the content of the specified cell number."""
    ipython = get_ipython()  # Get the current IPython instance
    
    # Ensure the cell number is valid
    if cell_number < len(ipython.user_ns['In']):
        prompt = "Please modify the code "  + ipython.user_ns['In'][cell_number] + " to accomplish " + modification + "Ignore the run cell magic and just focus on the code. Assume all library has been loaded. Return only code."
        response = model.generate_content(prompt)

        return create_new_cell("%%R\n\n" + response.text)

        # Return the content of the cell
        #return ipython.user_ns['In'][cell_number]
    else:
        # Error message for invalid cell number
        return "Cell number out of range."

def propose(suggest):
    response = model.generate_content("Please suggest code to accomplish " + suggest + ". Return only code.")
    #print(response.choices[0].message.content)
    #return display(Markdown(response.choices[0].message.content))
    return create_new_cell("\n\n" + response.text)
    
def create_new_cell(contents):
    shell = get_ipython()
    shell.set_next_input(contents, replace=False)


In [ ]:
propose('write me code for openai')

In [ ]:
propose("I need to find the 25 most signficantly upregulated genes from a dataframe called gene. Here are the column names: ENTREZID, SYMBOL, GENENAME, logFC AveExpr, t, P.Value, adj.P.Val")

In [ ]:
propose("help me develop a nextflow workflow")